# JSON Exercises

In [3]:
import pandas as pd
import json
from pandas.io.json import json_normalize 

## Read and Analyze JSON file

In [7]:
worldBanks = json.load((open('data/world_bank_projects.json')))
worldBanksdf = pd.read_json('data/world_bank_projects.json')  #load file into pandas 
worldBanksdf.head()  


,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17c783'},2014,October,2013-10-31T00:00:00Z,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,JB,IBRD,Active,N,"{'Percent': 50, 'Name': 'Participation and civ...","[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
4,{'$oid': '52b213b38594d8a2be17c784'},2014,October,2013-10-31T00:00:00Z,MINISTRY OF FINANCE,2019-04-30T00:00:00Z,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho,...,"FH,YW,YZ",IBRD,Active,N,"{'Percent': 30, 'Name': 'Export development an...","[{'code': '45', 'name': 'Export development an...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...


## Exercise 1.
1. Find the 10 countries with most projects

       - Count the number of times each country apears in 'countryname', take top 10

In [12]:
mostProjects = worldBanksdf['countryname'].value_counts().head(10)    
print('The 10 countries with the most projects are:\n')

print(mostProjects)

The 10 countries with the most projects are:

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64


## Exercise 2.
Find the top 10 major project themes (using column 'mjtheme_namecode')

In [17]:
#json_normalize the desired column to create data dataframe with code and name
themes = json_normalize(worldBanks, 'mjtheme_namecode')

#same as exercise 1, count the number of times name appears, take top 10
print('The 10 major project themes are:\n')
print(themes['name'].value_counts().head(10)) 


The 10 major project themes are:

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64


+ Notice that one of the top 10 themes is either null or empty string

## Exercise 3.
In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [25]:
#Identify the unique theme names
print(themes.name.unique(),'\n')

print(themes.code.unique())

['Human development' '' 'Economic management'
 'Social protection and risk management' 'Trade and integration'
 'Public sector governance' 'Environment and natural resources management'
 'Social dev/gender/inclusion' 'Financial and private sector development'
 'Rural development' 'Urban development' 'Rule of law'] 

['8' '11' '1' '6' '5' '2' '7' '4' '10' '9' '3']


In [33]:
#Get the index of the first time each name appears
uniqueNameIndexes = themes.name.drop_duplicates().index

#Create a lookup table that is indexed by code, excluding empty strings
lookup = themes.iloc[uniqueNameIndexes].loc[themes.name != ''].set_index('code')

print(lookup)

                                              name
code                                              
8                                Human development
11    Environment and natural resources management
1                              Economic management
6            Social protection and risk management
5                            Trade and integration
2                         Public sector governance
7                      Social dev/gender/inclusion
4         Financial and private sector development
10                               Rural development
9                                Urban development
3                                      Rule of law


In [34]:
#Map the name from the lookup table to the name column in themes
themes['name'] = themes['code'].map(lookup['name'])

#count the number of times name appears, take top 10
topThemes = themes['name'].value_counts().head(10)

print('The top 10 major themes are: \n')
print(topThemes)

The top 10 major themes are: 

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64
